# Experiment 1
This notebook contains the code for the first experiment of the paper. In this experiment, we train a model that first encodes the temporal dimensions of the EEG data and then its spatial dimensions. It does not include any B2B or classification heads.

In [1]:
%reload_ext autoreload
%autoreload 3

import pytorch_lightning as pl
from src.b2bnet import B2BNetSpaceTimeModel, OtkaTimeDimSplit

In [2]:
# Parameters
segment_size = 120 * 3 # 3sec
batch_size = 256
n_channels = 59
space_embedding_dim = 8
time_embedding_dim = 16
max_epochs = 10000

In [ ]:
# model params
data_mode = 'reconn'
b2b_data = 'hypnotist'
classifier = True
encoder_arch = 'encoder_only'
b2b = 'decoder'

datamodule = OtkaTimeDimSplit(segment_size=segment_size, batch_size=batch_size, b2b_data=b2b_data, data_mode=data_mode)

model = B2BNetSpaceTimeModel(
    n_channels=n_channels, space_embedding_dim=space_embedding_dim,
    time_embedding_dim=time_embedding_dim, kernel_size=1, n_subjects=52, classifier=classifier,
    encoder_arch=encoder_arch, b2b=b2b, data_mode=data_mode)

trainer = pl.Trainer(max_epochs=max_epochs, accelerator='cpu', log_every_n_steps=1, deterministic='warn')

trainer.fit(model, datamodule=datamodule)
# trainer.save_checkpoint(f'models/b2bnet_segment-{segment_size}_epochs-{trainer.current_epoch}.ckpt')